[Question Answering Minimal Start](https://simpletransformers.ai/docs/qa-minimal-start/)

In [1]:
%%capture

!pip install simpletransformers

In [2]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [4]:
import json
import requests

url = "https://raw.githubusercontent.com/josuemzx/Chatbot-para-refugiados/main/data/Dataset%20propio/data.json"
data = json.loads(requests.get(url).text)

In [5]:
data = [item for topic in data["data"] for item in topic['paragraphs'] ]

In [6]:
from sklearn.model_selection import train_test_split

train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)

In [7]:
train_data[0]["qas"][0]

{'question': '¿Cuál es el objetivo de la ley reguladora del derecho de asilo y de la protección subsidiaria?',
 'id': 618727,
 'answers': [{'answer_id': 662990,
   'document_id': 1288146,
   'question_id': 618727,
   'text': 'establecer los términos en que las personas nacionales de países no comunitarios y las apátridas podrán gozar en España de la protección internacional constituida por el derecho de asilo y la protección subsidiaria, así como el contenido de dicha protección internacional.',
   'answer_start': 149,
   'answer_end': 421,
   'answer_category': None}],
 'is_impossible': False}

In [8]:
eval_data[0]["qas"][0]

{'question': '¿Tengo derecho a abogado?',
 'id': 618672,
 'answers': [{'answer_id': 662935,
   'document_id': 1288144,
   'question_id': 618672,
   'text': 'Las personas solicitantes de asilo tienen derecho a entrevistarse con un abogado en las dependencias de los puestos fronterizos y centros de internamiento de extranjeros.',
   'answer_start': 1107,
   'answer_end': 1277,
   'answer_category': None}],
 'is_impossible': False}

In [9]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True

import shutil

output_directory = "outputs"
shutil.rmtree(output_directory, ignore_errors=True)

model = QuestionAnsweringModel(
    "roberta", "roberta-base", args=model_args
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

### Probar el modelo original antes de entrenarlo con nuestros datos

In [10]:
def prediction(context, question, limit=5):
  to_predict = [
      {
          "context": context,
          "qas": [
              {
                  "question": question,
                  "id": "0",
              }
          ],
      }
  ]

  answers, probabilities = model.predict(to_predict)

  answers = answers[0]["answer"][:limit]
  probabilities = probabilities[0]["probability"][:limit]

  for (answer, probability) in zip(answers, probabilities):
    print(f"{round(probability,2)}: {answer}")

In [11]:
contexto_1 = """
Derecho a solicitar protección internacional.\n\n1. Las personas nacionales no comunitarias
y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.\n\n2.
Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asistencia sanitaria y a asistencia
jurídica gratuita, que se extenderá a la formalización de la solicitud y a toda la tramitación del procedimiento,
y que se prestará en los términos previstos en la legislación española en esta materia, así como derecho a intérprete
en los términos del artículo 22 de la Ley Orgánica 4/2000.\n\nLa asistencia jurídica referida en el párrafo anterior
será preceptiva cuando las solicitudes se formalicen de acuerdo al procedimiento señalado en el artículo 21 de la
presente Ley.\n\n3. La presentación de la solicitud conllevará la valoración de las circunstancias determinantes
del reconocimiento de la condición de refugiado, así como de la concesión de la protección subsidiaria.
De este extremo se informará en debida forma al solicitante.\n\n4. Toda información relativa al procedimiento,
incluido el hecho de la presentación de la solicitud, tendrá carácter confidencial.
"""

pregunta_1 = "¿Puedo solicitar traductor?"

prediction(contexto_1, pregunta_1)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 1704.31it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0.06: 1. Las personas nacionales no comunitarias y las apátridas presentes
0.05: 1. Las personas nacionales
0.05: 1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar
0.05: 1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España. 2. Para su ejercicio, los solicitantes de protección internacional tendrán derecho a asistencia
0.05: 1. Las personas nacionales no comunitarias y las


In [12]:
contexto_2 = """
Las subvenciones concedidas a las ONG para la acogida y la integración de quienes conforman estos colectivos
están financiadas con fondos que provienen de los presupuestos de la Secretaría General de Inmigración y
Emigración, recibiendo cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.
"""

pregunta_2 = "donde solicito las subenciones?"

prediction(contexto_2, pregunta_2)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 12671.61it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0.05: integración de quienes conforman estos colectivos
0.05: integración de quienes conforman estos colectivos están financiadas
0.05: estos colectivos
0.05: estos colectivos están financiadas
0.05: conforman estos colectivos


### Entrenar el modelo con nuestros datos

In [13]:
# Train the model
model.train_model(train_data, eval_data=eval_data)

add example index and unique id: 100%|██████████| 84/84 [00:00<00:00, 284589.29it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/9 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

convert squad examples to features: 100%|██████████| 13/13 [00:00<00:00, 49.09it/s]

add example index and unique id: 100%|██████████| 13/13 [00:00<00:00, 110153.44it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

(9,
 {'global_step': [9],
  'correct': [0],
  'similar': [12],
  'incorrect': [1],
  'train_loss': [4.151611328125],
  'eval_loss': [-1.3401692708333333]})

In [14]:
# Evaluate the model
result, texts = model.eval_model(eval_data)

add example index and unique id: 100%|██████████| 13/13 [00:00<00:00, 98422.30it/s]


Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

### Probar el nuevo modelo

In [15]:
prediction(contexto_1, pregunta_1)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 555.83it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0.12: .
0.12: internacional. 1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.
0.12: 
0.05: Derecho a solicitar protección internacional.
0.05: Derecho a solicitar protección internacional. 1. Las personas nacionales no comunitarias y las apátridas presentes en territorio español tienen derecho a solicitar protección internacional en España.


In [16]:
prediction(contexto_2, pregunta_2)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2236.96it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

0.17: .
0.16: 
0.05: Europeo y del Fondo de Asilo, Migración e Integración.
0.05: estos colectivos están financiadas con fondos que provienen de los presupuestos de la Secretaría General de Inmigración y Emigración, recibiendo cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.
0.05: financiadas con fondos que provienen de los presupuestos de la Secretaría General de Inmigración y Emigración, recibiendo cofinanciación, en algunos programas, del Fondo Social Europeo y del Fondo de Asilo, Migración e Integración.
